# Linear Regression with sklearn

In [1]:
import pandas as pd
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 1. load data
X, y = datasets.load_diabetes(return_X_y=True)
X = X[:,1:3]

# 2. Split Train and Test Data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=5
)

# 3. Fit model
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

# 4. Predict
y_pred = reg.predict(X_test)
print(f"MSE: {mean_squared_error(y_test, y_pred)}")
print(reg.coef_)

MSE: 4392.632885400543
[  2.33410605 938.42795975]


# Linear Regression from Scratch

## Components
### 1. Prediction
### 2. Training

## Prediction
### Mathematical formula

$$
\begin{align}
y=\theta^Tx
\end{align}
$$
$$
\begin{align}
h_{\theta}(x)=\theta^Tx
\end{align}
$$

- notations
    - $\theta$: weight vector(D x 1 dim.)
    - **x**: input(1 row) vector(D x 1 dim.)
    - D: the number of features + 1
- 위 둘은 같은 식
- 참고
    - 보통 선형회귀식은 intercept 항을 따로 빼서 표기하기지만, 이후 수식 전개 등을 쉽게 하기 위해 theta0가 intercept라고 둠.

In [2]:
import numpy as np
N = X_train.shape[0]
D = X_train.shape[1]+1
theta = np.random.normal(0, 1, D).reshape(-1,1)
X = np.concatenate([np.ones(X_train.shape[0]).reshape(-1,1), X_train], axis=1) # Add bias term
x = X_train[0,:]
x = np.concatenate([[1], x]).reshape(-1,1) # add 1 for intercept 

# using for loop
prediction = 0
for i in range(D):
    prediction += theta[i][0]*x[i][0]
    prediction = prediction

# vectorized
prediction = theta.T.dot(x)

# Predict multiple data
predictions = X.dot(theta)

def predict(X, theta):
    X = np.concatenate([np.ones(X.shape[0]).reshape(-1,1), X], axis=1) # Add bias term
    y_hat = X.dot(theta)
    
    return y_hat.flatten()

## Training
### Cost function(= Loss function)
#### Cost function of linear regression: Mean Squared Error

$$
\begin{align}
J(\theta) = \frac{1}{N}\sum_{i=1}^N(h_{\theta}(x^{(i)})-y^{(i)})^2
\end{align}
$$
$$
\begin{align}
J(\theta) = \frac{1}{N}\sum_{i=1}^N((\theta^Tx^{(i)}+b)-y^{(i)})^2
\end{align}
$$
$$
\begin{align}
J(\theta) = \frac{1}{N}(X\theta-y)^T(X\theta-y)
\end{align}
$$

- notations
    - **$\theta$**: weight vector(D x 1 dim.)
    - **x**: input(1 row) vector(D x 1 dim.)
    - **X**: input matrix(N x D dim.)
    - **y**: output vector(N x 1 dim.)
    - D: the number of features
- 위 세 개는 전부 같은 식
- MAE를 최소가 되게 하는 **w**를 찾는다!

In [3]:
# cost function
def mse(y_hat, y):
    N = len(y)
    cost = sum((y_hat-y)**2)/N
    return cost

## Optimization Method
- 위에 정의한 cost function을 최대화 하기위해서는 어떠한 과정이 필요한가?
- 선형회귀의 경우, 미분을 사용해서 해를 구할 수 있음!
- 단, 일반적으로는 수리적인 방법(Ex. 동전던지기의 확률을 구한 방법, 미분 등)으로 해를 구할 수 없음
- Optimization Method의 대표적인 예: **Gradient descent**
    - Mathematical Formula
$$
\begin{align}
\theta \leftarrow \theta - \eta{\nabla}_{\theta}J
\end{align}
$$
$$
\begin{align}
{\nabla}_{\theta}J=\sum_{n=1}^Nx^{(i)}(h_{\theta}(x^{(i)})-y^{(i)})
\end{align}
$$
$$
\begin{align}
{\nabla}_{\theta}J=\frac{1}{N}X^T(X\theta-y)
\end{align}
$$
</br>
$$
h_{\theta}(x^{(i)}) = \theta_0 + \theta_1x_1 + \theta_2x_2+...+ \theta_Dx_D
$$

- notations
    - $\eta$ : learning rate
 
<img src="../figures/GradientDescentGIF.gif" alt="drawing" width="600" align="center"/>

--------------------

## 조금 더 깊게

- 기계학습의 “학습”은 단순히 모델의 가중치(w)를 찾아내는 것
    - 비유하자면, 새로운 기억이 생성될 때마다, 뇌에 있는 각 시냅스 간의 연결의 세기가 변한다!
- 이러한 관점에서, 기계학습 문제는 단순히 주어진 데이터(X, y)를 가장 잘 설명하는 가중치를 찾아내는 것이다.
- 이러한 가중치를 찾아내는 방법 중 가장 많이 사용되는 것이 최대우도추정(Maximum likelihood Estimation) 방법이다. 

### Likelihood?
<!-- ![likelihoood](../figures/likelihood2.png) -->
<img src="../figures/likelihood2.png" alt="drawing" width="600"/>

### Base theorem
<img src="../figures/baise_theorem.png" alt="drawing" width="400"/>


In [4]:
# Gradient Descent 함수 생성
def gradientDescent(X, y, theta, alpha, N, numIterations, verbose=1):
    X_tmp = X.copy()
    X_tmp = np.concatenate([np.ones(X_tmp.shape[0]).reshape(-1,1), X_tmp], axis=1) # Add bias term
#     tmp = (tmp - tmp.mean()) / (tmp.max() - tmp.min()) # standardization for computing convinience
    # bias 추가
    for i in range(0, numIterations):
        # Predict
        hypothesis = X_tmp.dot(theta)
        loss = hypothesis.reshape(-1,1) - y.reshape(-1,1)
        # avg cost per example (the 2 in 2*n doesn't really matter here.
        # But to be consistent with the gradient, I include it)
        cost = np.sum(loss ** 2) / N
        if verbose==1:
            if(i%100000==0):
                print("Iteration %d | Cost: %f" % (i, cost))
        # avg gradient per example
        gradient = X_tmp.T.dot(loss) / N
        # update
        theta = theta - alpha * gradient
    return theta

# Linear Regression with Numpy

In [5]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def predict(X, theta):
    X = np.concatenate([np.ones(X.shape[0]).reshape(-1,1), X], axis=1) # Add bias term
    y_hat = X.dot(theta)
    
    return y_hat.flatten()

# Gradient Descent 함수 생성
def gradientDescent(X, y, theta, alpha, N, numIterations, verbose=1):
    X_tmp = X.copy()
    X_tmp = np.concatenate([np.ones(X_tmp.shape[0]).reshape(-1,1), X_tmp], axis=1) # Add bias term
#     tmp = (tmp - tmp.mean()) / (tmp.max() - tmp.min()) # standardization for computing convinience
    # bias 추가
    for i in range(0, numIterations):
        # Predict
        hypothesis = X_tmp.dot(theta)
        loss = hypothesis.reshape(-1,1) - y.reshape(-1,1)
        # avg cost per example (the 2 in 2*n doesn't really matter here.
        # But to be consistent with the gradient, I include it)
        cost = np.sum(loss ** 2) / N
        if verbose==1:
            if(i%100000==0):
                print("Iteration %d | Cost: %f" % (i, cost))
        # avg gradient per example
        gradient = X_tmp.T.dot(loss) / N
        # update
        theta = theta - alpha * gradient
    return theta

# 1. load data
X, y = datasets.load_diabetes(return_X_y=True)
X = X[:,1:3]
D = X_train.shape[1]+1

# 2. Split Train and Test Data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=5
)

# 3. Initialize model
# - Set hyper parameters
epochs= 500000
learning_rate = 0.005
# - Initialize weight
theta = np.random.normal(0, 1, D).reshape(-1,1) #Initalize theta

# 4. Fit model
theta = gradientDescent(X=X_train,
                        y=y_train,
                        theta=theta,
                        alpha=learning_rate,
                        N=X_train.shape[0],
                        numIterations=epochs,
                        verbose=1)
# 4. Predict
y_pred = predict(X_test, theta)
print(f"MSE: {mean_squared_error(y_test, y_pred)}")

Iteration 0 | Cost: 28779.649938
Iteration 100000 | Cost: 3963.942000
Iteration 200000 | Cost: 3784.199511
Iteration 300000 | Cost: 3766.496319
Iteration 400000 | Cost: 3764.711608
MSE: 4393.753715842214


In [6]:
print("Compare coefficients. sklearn vs. mine")
print(pd.DataFrame({"sklearn":np.concatenate([[reg.intercept_],reg.coef_.flatten()]), "mine":theta.flatten()}))

Compare coefficients. sklearn vs. mine
      sklearn        mine
0  151.812188  151.810914
1    2.334106    3.472117
2  938.427960  935.514359


# Linear Regression with Pytorch 1

In [7]:
import torch
import torch.optim as optim

import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def predict(X, theta):
    hypothesis = torch.matmul(X, theta)
    
    return hypothesis

# 1. load data
X, y = datasets.load_diabetes(return_X_y=True)
X = X[:,1:3]
X = np.concatenate([np.ones(X.shape[0]).reshape(-1,1), X], axis=1) # Add bias term

# 2. Split Train and Test Data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=5
)
N = X_train.shape[0]
D = X_train.shape[1]

X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float()
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float()

# 3. Initialize model
# - Set hyper parameters
epochs= 500000
learning_rate = 0.005
# - Initialize weight
theta = torch.zeros(D, requires_grad=True).float()
# - Set optimizezr
optimizer = optim.SGD([theta], lr=learning_rate) # Stochastic Gradient Descenct

# 4. Fit model
for epoch in range(epochs):
    # H(x) 계산
    hypothesis = torch.matmul(X_train, theta)
#     hypothesis = torch.dot(X_train,theta.view(-1,1))

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100000 == 0:
        print(f"Epoch {epoch}/{epochs} Cost: {cost.item()}")
        
# 5. Predict
y_pred = predict(X_test, theta)
print(f"MSE: {mean_squared_error(y_test, y_pred.detach().numpy())}")

Epoch 0/500000 Cost: 28527.291015625
Epoch 100000/500000 Cost: 3784.14306640625
Epoch 200000/500000 Cost: 3764.7109375
Epoch 300000/500000 Cost: 3764.510009765625
Epoch 400000/500000 Cost: 3764.509521484375
MSE: 4392.9892578125


In [8]:
print("Compare coefficients. sklearn vs. mine")
print(pd.DataFrame({"sklearn":np.concatenate([[reg.intercept_],reg.coef_.flatten()]), "mine":theta.detach().numpy()}))

Compare coefficients. sklearn vs. mine
      sklearn        mine
0  151.812188  151.811111
1    2.334106    2.425052
2  938.427960  937.117493


# Linear Regression with Pytorch 2

In [9]:
# 1. load data
X, y = datasets.load_diabetes(return_X_y=True)
X = X[:,1:3]
# X = np.concatenate([np.ones(X.shape[0]).reshape(-1,1), X], axis=1)

# 2. Split Train and Test Data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=5
)
N = X_train.shape[0]
D_ = X_train.shape[1]

X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float().view(-1,1)
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float().view(-1,1)

# 3. Initialize model
class LinearRegression(torch.nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = torch.nn.Linear(D_, 1) # bias term is automatically added.
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred

epochs= 500000
learning_rate = 0.005

model = LinearRegression()
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 4. Fit model
for epoch in range(epochs):
    model.train()
    # Forward pass
    y_pred = model(X_train)
    # Compute Loss
    loss = criterion(y_pred, y_train)
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    

# 5. Predict
y_pred = model(X_test)
print(f"MSE: {mean_squared_error(y_test, y_pred.detach().numpy())}")

MSE: 4392.9892578125


In [10]:
print("Compare coefficients. sklearn vs. mine")
theta1_ = model.linear.weight.detach().numpy().flatten() # get weigths from model.
theta0 = model.linear.bias.detach().numpy().flatten() # get bias(intercept) weight from model.
theta = np.concatenate([theta0,theta1_]) # concat weights
print(pd.DataFrame({"sklearn":np.concatenate([[reg.intercept_],reg.coef_.flatten()]), "mine":theta}))

Compare coefficients. sklearn vs. mine
      sklearn        mine
0  151.812188  151.811111
1    2.334106    2.425158
2  938.427960  937.117493


# References
likelihood1: https://jjangjjong.tistory.com/41  
likelihood2: https://angeloyeo.github.io/2020/07/17/MLE.html  
cost function: https://computer-nerd.tistory.com/5  
Deriving Machine Learning Cost Functions using Maximum Likelihood Estimation: https://allenkunle.me/deriving-ml-cost-functions-part1  
Linear Regression Normality: https://stats.stackexchange.com/questions/327427/how-is-y-normally-distributed-in-linear-regression  
gradient descent: https://mccormickml.com/2014/03/04/gradient-descent-derivation/  
notatinos: https://humanunsupervised.github.io/humanunsupervised.com/topics/L2-linear-regression-multivariate.html  
linear regression with pytorch1: https://wikidocs.net/53560  
linear regression with pytorch2: https://medium.com/biaslyai/pytorch-linear-and-logistic-regression-models-5c5f0da2cb9  